# Анализ зависимости Q-values от P&L

Этот ноутбук загружает пары сделок BUY→SELL c сервера (`/api/analysis/qvalues_vs_pnl`), строит распределения, калибровку и подбирает пороги T1/T2 для maxQ и gapQ, чтобы улучшить фильтрацию входов (Q-gate).

- buy_max_q: максимум из q_values в момент BUY
- buy_gap_q: разница между 1-й и 2-й по величине q_values (разрыв уверенности)
- Метка успеха: win = pnl_abs > 0 (можно сменить порог на pnl_pct)



In [ ]:
import os
import json
import math
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve

BASE_URL = os.environ.get('MEDOED_BASE_URL', 'http://localhost:5050')
SYMBOL = os.environ.get('ANALYSIS_SYMBOL', '')  # например 'BTCUSDT'
LIMIT_TRADES = int(os.environ.get('ANALYSIS_LIMIT_TRADES', '400'))
LIMIT_PREDS = int(os.environ.get('ANALYSIS_LIMIT_PREDS', '2000'))
TOLERANCE = int(os.environ.get('ANALYSIS_TOLERANCE', '1'))

params = {
    'symbol': SYMBOL,
    'limit_trades': LIMIT_TRADES,
    'limit_predictions': LIMIT_PREDS,
    'tolerance_buckets': TOLERANCE,
}
resp = requests.get(f"{BASE_URL}/api/analysis/qvalues_vs_pnl", params=params)
resp.raise_for_status()
js = resp.json()
assert js.get('success'), js
rows = js.get('rows', [])
df = pd.DataFrame(rows)
print(df.shape)
df.head()


In [ ]:
# Предобработка

df['win'] = (df['pnl_abs'] > 0).astype(int)
df['buy_max_q'] = pd.to_numeric(df['buy_max_q'], errors='coerce')
df['buy_gap_q'] = pd.to_numeric(df['buy_gap_q'], errors='coerce')
df = df.dropna(subset=['buy_max_q'])
print('wins:', df['win'].sum(), 'losses:', (1 - df['win']).sum())
df[['symbol','entry_time','exit_time','pnl_abs','buy_max_q','buy_gap_q','win']].head()


In [ ]:
# Гистограммы распределений

fig, axs = plt.subplots(1,2, figsize=(12,4))
axs[0].hist(df[df.win==1]['buy_max_q'], bins=30, alpha=0.7, label='win')
axs[0].hist(df[df.win==0]['buy_max_q'], bins=30, alpha=0.7, label='loss')
axs[0].set_title('buy_max_q by class'); axs[0].legend()

axs[1].hist(df[df.win==1]['buy_gap_q'].dropna(), bins=30, alpha=0.7, label='win')
axs[1].hist(df[df.win==0]['buy_gap_q'].dropna(), bins=30, alpha=0.7, label='loss')
axs[1].set_title('buy_gap_q by class'); axs[1].legend()
plt.show()


In [ ]:
# ROC/PR для buy_max_q как шкалы уверенности

scores = df['buy_max_q'].values
labels = df['win'].values
fpr, tpr, thr = roc_curve(labels, scores)
roc_auc = auc(fpr, tpr)
prec, rec, thr_pr = precision_recall_curve(labels, scores)

fig, axs = plt.subplots(1,2, figsize=(12,4))
axs[0].plot(fpr, tpr, label=f'AUC={roc_auc:.3f}')
axs[0].plot([0,1],[0,1],'--',color='gray')
axs[0].set_title('ROC (buy_max_q)'); axs[0].legend()
axs[0].set_xlabel('FPR'); axs[0].set_ylabel('TPR')
axs[1].plot(rec, prec)
axs[1].set_title('PR (buy_max_q)'); axs[1].set_xlabel('Recall'); axs[1].set_ylabel('Precision')
plt.show()


In [ ]:
# Калибровка: вероятность win по бинам buy_max_q

bins = np.quantile(df['buy_max_q'], np.linspace(0,1,11))
bins[0] = df['buy_max_q'].min() - 1e-9
bins[-1] = df['buy_max_q'].max() + 1e-9
labels_bins = pd.cut(df['buy_max_q'], bins=bins, include_lowest=True)
calib = df.groupby(labels_bins)['win'].mean().reset_index()
calib['mid'] = calib['buy_max_q'].apply(lambda x: x.mid)
calib = calib.dropna(subset=['mid'])
plt.figure(figsize=(6,4))
plt.plot(calib['mid'], calib['win'], marker='o')
plt.title('Калибровка: P(win) vs buy_max_q')
plt.xlabel('buy_max_q'); plt.ylabel('P(win)')
plt.grid(True)
plt.show()
calib


In [ ]:
# Автоподбор T1/T2 по сетке

best = None
maxq_vals = np.quantile(df['buy_max_q'], np.linspace(0.2, 0.9, 15))
gapq_vals = np.quantile(df['buy_gap_q'].dropna(), np.linspace(0.2, 0.9, 15)) if df['buy_gap_q'].notna().any() else [0.0]

for t1 in maxq_vals:
    for t2 in gapq_vals:
        mask = (df['buy_max_q'] >= t1)
        if df['buy_gap_q'].notna().any():
            mask &= (df['buy_gap_q'] >= t2)
        sub = df[mask]
        if len(sub) < 20:
            continue
        hit = sub['win'].mean()
        # простая метрика: precision на отобранных + объем выборки
        score = hit * (len(sub) / max(1, len(df)))
        if (best is None) or (score > best['score']):
            best = {'T1': float(t1), 'T2': float(t2), 'hit_rate': float(hit), 'n': int(len(sub)), 'score': float(score)}

best


In [ ]:
# Экспорт найденных порогов в .env строку для агента

if best:
    print('Рекомендуемые пороги:')
    print(best)
    print('\nДля запуска агента с фильтром:')
    print(f"QGATE_MAXQ={best['T1']:.6f} QGATE_GAPQ={best['T2']:.6f}")
else:
    print('Недостаточно данных для подбора порогов')
